In [1]:
import pandas as pd

In [ ]:
original_train_embed

In [ ]:
all_features = ["tweet_id" , "tweet_timestamp"]
train = pd.read_csv("s3://recsys-challenge-2020/training.tsv", encoding="utf-8",
                    names = all_features, usecols= [2, 8], sep="\x01"
                   )
train_sorted = train.sort_values(by=['tweet_timestamp']).reset_index()
n_head = 90
n_tail = 10
train_set = train_sorted.head(int(len(train_sorted)*(n_head/100)))
val_set = train_sorted.tail(int(len(train_sorted)*(n_tail/100)))
lang_embedd_train_set = pd.read_csv('s3://recsys-challenge-2020/fast_text_embeddings/lang_embedd_train_set.csv', header = None)
lang_embed_val_set = pd.read_csv('s3://recsys-challenge-2020/fast_text_embeddings/lang_embedd_val_set.csv', header = None)
train_embed_set = pd.concat([train_set, lang_embedd_train_set], axis = 1)
val_embed_set = pd.concat([val_set, lang_embed_val_set], axis = 1)
train_embed = pd.concat([train_embed_set, val_embed_set])
original_train_embed = pd.merge(train, train_embed, how = 'left', left_on = 'tweet_id', right_on = 'tweet_id')